<a href="https://colab.research.google.com/github/agatiks/Methods-of-optimization/blob/main/lab2_StochasticGradientDescent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

Возьмём датасет, содержащий 200 записей о росте (в дюймах) и весе (в фунтах) 18-летних людей с этого сайта http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_Dinov_020108_HeightsWeights. На нём и будем решать задачу линейной регрессии.




In [1]:
import numpy as np
import matplotlib.pyplot as plt
!pip install memory_profiler
import memory_profiler
%load_ext memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=081ef9953c2b6e98bcf1fee913a945b7abf4f939f2e4201d7d00098c38e7e369
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


In [2]:
file_wh = "file_1.txt" #https://drive.google.com/file/d/1TeLCr1wtKUKHefQWoMb3OEV-XZQeIwoE/view?usp=sharing

In [3]:
def get_wh_points():
  f = open(file_wh, 'r')

  points_wh = []
  for f_str in f.readlines():
    if not f_str.isspace():
      _, x, y = map(float, f_str.split())
      points_wh.append(np.array([x, y]))
  return points_wh


def get_simple_points(size = 30):
  points = []
  try: 
    f = open("simple_file.txt", 'r')
    for f_str in f.readlines():
      if not f_str.isspace():
        _, x, y = map(float, f_str.split())
        points.append(np.array([x, y]))
  except:
    for i in range(size):
      points.append(np.array([i / 10, i / 10]))
  return points

def get_simple_points_with_up(size = 30):
  points = []
  try:
    f = open("simple_points_with_up.txt", 'r')
    for f_str in f.readlines():
      if not f_str.isspace():
        _, x, y = map(float, f_str.split())
        points.append(np.array([x, y]))
  except:
    for i in range(size - 1):
      points.append(np.array([i / 10, i / 10]))
    points.append(np.array([(size - 1) / 10, random.random() * 2 - 1]))
  return points


In [4]:
def show_points(points, title=""):
  x = [p[0] for p in points]
  y = [p[1] for p in points]
  
  plt.scatter(x, y, s = 10)
  plt.title(title)
  plt.show()


def show_points_and_line(X, Y, points, title=""):
  x = [p[0] for p in points]
  y = [p[1] for p in points]
  
  plt.scatter(x, y, s = 10)
  plt.title(title)
  plt.plot(X, Y, color="g")
  plt.show()



In [5]:
try:
  show_points(get_wh_points(), "Зависимость веса и роста 18-летних людей")
except FileNotFoundError:
  print("File wasn't uploaded")

File wasn't uploaded


Будем подбирать параметры a и b линейной функции a * x + b = y, минимизируя сумму наименьших квадратов ошибок - MSE методом градиентного спуска

In [6]:
def MSE(true_points, coeffs):
  sum = 0
  a = coeffs[0]
  b = coeffs[1]
  for point in true_points:
    sum += pow((point[1] - (point[0] * a + b)), 2) 
  return sum 

# её градиент
def grad_MSE(points, coeffs, batch_size):
  sum_a = 0
  sum_b = 0
  dataset_size = len(points)
  if batch_size == dataset_size:
    for i in range(batch_size):
      x_i = points[i][0]
      y_i = points[i][1]
      sum_a += -2 * x_i * (y_i - (coeffs[0] * x_i + coeffs[1]))
      sum_b += -2 * (y_i - (coeffs[0] * x_i + coeffs[1]))
  else:
    j = 0
    while j < batch_size:
      i = random.randint(0, dataset_size - 1)
      x_i = points[i][0]
      y_i = points[i][1]
      sum_a += -2 * x_i * (y_i - (coeffs[0] * x_i + coeffs[1]))
      sum_b += -2 * (y_i - (coeffs[0] * x_i + coeffs[1]))
      j += 1
    

  '''print(sum_a, sum_b, "<- sums")'''
  return np.array([sum_a , sum_b])


Сама реализация стохастического градиентного спуска (для начала с размером батча = n, то есть обычного градиентного спуска):

In [7]:
import random
const_lr = 0.0000002

EPS = 1E-4
def shocasticGD(batch_size, points, start_coeffs, lr=const_lr, epoch = 300):
    i = 0
    f = True
    coeffs = start_coeffs
    print(MSE(points, coeffs), "<- start MSE")
    while i < epoch:
      prev_coeffs = coeffs.copy()
      coeffs -= lr * grad_MSE(points, prev_coeffs, batch_size)
      '''print("res coefs for i =", i, coeffs)'''
      random.shuffle(points)
      i += 1
      if(abs(MSE(points, coeffs) - MSE(points, prev_coeffs)) < EPS and f):
        print("{i} epochs were enough.".format(i = i))
        f = False
    print("All {i} epochs were done.".format(i = epoch))
    print(MSE(points, coeffs), "<- min MSE")
    res_coeffs = coeffs
    return coeffs
wh_points = get_wh_points()
res_coeffs = shocasticGD(len(wh_points), wh_points, np.array([0.1, 0.1]))
print(res_coeffs, "<- res")

FileNotFoundError: ignored

In [ ]:
X = []
Y = []
for i in get_wh_points():
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, get_wh_points(), "Зависимость веса и роста 18-летних людей")
except FileNotFoundError:
  print("File wasn't uploaded")

Работа нашего алгоритма для точек на прямой y = x: (увеличили lr и уменьшили кол-во эпох)

In [ ]:
X = []
Y = []
simple_points = get_simple_points()
res_coeffs = shocasticGD(len(simple_points),simple_points, np.array([0.1, 0.1]), 0.002, 100)
for i in simple_points:
  X.append(i[0])
  Y.append(res_coeffs[0] * i[0] + res_coeffs[1])


try:
  show_points_and_line(X, Y, simple_points, "Прямая y = x")
except FileNotFoundError:
  print("File wasn't uploaded")

Теперь для точек располагающихся на прямой y = x с резким выбросом одной точки:

In [ ]:
X = []
Y = []
simple_points_with_up = get_simple_points_with_up()
res_coeffs = shocasticGD(len(simple_points_with_up), simple_points_with_up, np.array([0.1, 0.1]), 0.00002, 100)
for i in simple_points_with_up:
  X.append(i[0])
  Y.append(res_coeffs[0] * i[0] + res_coeffs[1])


try:
  show_points_and_line(X, Y, simple_points_with_up, "Прямая y = x с отдельной точкой")
except FileNotFoundError:
  print("File wasn't uploaded")

Теперь возьмём размер батча = 1:

In [ ]:
wh_points = get_wh_points()
res_coeffs = shocasticGD(1, wh_points, np.array([0.1, 0.1]), epoch = 300)
X = []
Y = []
for i in get_wh_points():
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, get_wh_points(), "Зависимость веса и роста 18-летних людей")
except FileNotFoundError:
  print("File wasn't uploaded")

Теперь MiniBatch (n / 2):

In [ ]:
wh_points = get_wh_points()
res_coeffs = shocasticGD(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
X = []
Y = []
for i in get_wh_points():
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, get_wh_points(), "Зависимость веса и роста 18-летних людей")
except FileNotFoundError:
  print("File wasn't uploaded")

Видим, что результат линейной регрессии для размера батча = 1, вообще мало похож на правду, в то время как для miniBatch результат приближен к nGd, хотя кол-во вычислений функции в 2 раза меньше, что даёт сделать вывод о том, что miniBatch реализация является наиболее оптимальной.

# Задание 2

Теперь нормальзуем данные:

z_i = beta * z_i-1 + (1 - beta) * y_i

И на них запустим наш SGD


In [ ]:
points_wh = get_wh_points()
#points_wh = points_wh/np.max(points_wh)
print(points_wh)
beta = 0.6
for i in range(1, len(points_wh)):
  y_prev = points_wh[i - 1][1]
  y_i = points_wh[i][1]
  points_wh[i][1] = beta * y_prev + (1 - beta) * y_i


res_coeffs = shocasticGD(len(points_wh) // 2, points_wh, np.array([0.1, 0.1]))
X = []
Y = []
for i in get_wh_points():
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, points_wh, "Зависимость веса и роста 18-летних людей")
except FileNotFoundError:
  print("File wasn't uploaded")




Видим, что предварительная нормализация данных позволила сильно уменьшить результат функции ошибки MSE (с тем же кол-вом эпох и lr):

с нормализацией: 1869.7431200437848 <- min MSE

без: 6148.783264426831 <- min MSE


# Задание 3


(в этих модификациях метода стохастического градиентного спуска будем работать с miniBatch(n / 2), т.к. уже сделали вывод, что это самый оптимальный размер батча)

SGD with momentum:

In [ ]:
EPS = 1e-3
def stochasticGDmomentum_all(batch_size, points, start_coeffs, lr=const_lr, epoch = 100):
  i = 0
  coeffs = start_coeffs
  print(MSE(points, coeffs), "<- start MSE")
  td_prev=coeffs
  beta = 0.8
  f = True
  mse = []
  list_coeffs = []
  while i < epoch:
    list_coeffs.append(coeffs.copy())
    mse.append(MSE(points, coeffs))
    prev_coeffs = coeffs
    td = beta * td_prev + lr * grad_MSE(points, prev_coeffs, batch_size)
    td_prev = td
    coeffs -= td
    random.shuffle(points)
    #print(MSE(points, coeffs))
    i += 1
    if(abs(MSE(points, coeffs) - MSE(points, prev_coeffs)) < EPS and f):
      print("{i} epochs were enough.".format(i = i))
      f = False
  if (not (i < epoch)):
    print("All {i} epochs were done.".format(i = epoch))
  print(MSE(points, coeffs), "<- min MSE")
  fig = plt.figure()
  ax1 = fig.add_subplot(211)
  ax1.set_xlabel('epoch')
  ax1.plot(np.arange(epoch), mse)
  plt.show()
  res_coeffs = coeffs
  return list_coeffs

def stochasticGDmomentum(batch_size, points, start_coeffs, lr=const_lr, epoch = 100):
  return stochasticGDmomentum_all(batch_size, points, start_coeffs, lr, epoch)[-1]

wh_points = get_wh_points()
res_coeffs = stochasticGDmomentum(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
X = []
Y = []
for i in get_wh_points():
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, get_wh_points(), "Зависимость веса и роста 18-летних людей momentum")
except FileNotFoundError:
  print("File wasn't uploaded")

Nesterov:

In [ ]:
def stochasticGDnesterov_all(batch_size, points, start_coeffs, lr=const_lr, epoch = 100):
  i = 0
  coeffs = start_coeffs
  print(MSE(points, coeffs), "<- start MSE")
  td_prev=coeffs
  beta = 0.8
  f = True
  mse = []
  list_coeffs = []
  while i < epoch:
    list_coeffs.append(coeffs.copy())
    mse.append(MSE(points, coeffs))
    prev_coeffs = coeffs
    for j in points:
      j -= beta * td_prev
    td = beta * td_prev + lr * grad_MSE(points, prev_coeffs, batch_size)
    td_prev = td
    coeffs -= td
    '''print("res coefs for i =", i, coeffs)'''
    random.shuffle(points)
    i += 1
    if(abs(MSE(points, coeffs) - MSE(points, prev_coeffs)) < EPS and f):
      print("{i} epochs were enough.".format(i = i))
      f = False
  fig = plt.figure()
  ax1 = fig.add_subplot(211)
  ax1.set_xlabel('epoch')
  ax1.plot(np.arange(epoch), mse)
  plt.show()
  if (not (i < epoch)):
    print("All {i} epochs were done.".format(i = epoch))
  print(MSE(points, coeffs), "<- min MSE")
  res_coeffs = coeffs
  return list_coeffs
def stochasticGDnesterov(batch_size, points, start_coeffs, lr=const_lr, epoch = 100):
  return stochasticGDnesterov_all(batch_size, points, start_coeffs, lr, epoch)[-1]

In [ ]:
wh_points = get_wh_points()
res_coeffs = stochasticGDnesterov(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
X = []
Y = []
for i in get_wh_points():
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, get_wh_points(), "Зависимость веса и роста 18-летних людей momentum")
except FileNotFoundError:
  print("File wasn't uploaded")

AdaGrad (адаптивный градиентный алгоритм):

In [ ]:
def stochasticGDAdaGrad_all(batch_size, points, start_coeffs, lr=const_lr, epoch = 100):
    i = 0
    coeffs = start_coeffs
    print(MSE(points, coeffs), "<- start MSE")
    sum_prev=coeffs
    mse = []
    list_coeffs = []
    while i < epoch:
      list_coeffs.append(coeffs.copy())
      mse.append(MSE(points, coeffs))
      prev_coeffs = coeffs
      sum = sum_prev + np.square(grad_MSE(points, prev_coeffs, batch_size))
      sum_prev = sum
      coeffs -= lr * np.divide(grad_MSE(points, prev_coeffs, batch_size), np.sqrt(sum))
      random.shuffle(points)
      i += 1
    print(MSE(points, coeffs), "<- min MSE")
    fig = plt.figure()
    ax1 = fig.add_subplot(211)
    ax1.set_xlabel('epoch')
    ax1.plot(np.arange(epoch), mse)
    plt.show()
    res_coeffs = coeffs
    return list_coeffs
def stochasticGDAdaGrad(batch_size, points, start_coeffs, lr=const_lr, epoch = 100):
  return stochasticGDAdaGrad_all(batch_size, points, start_coeffs, lr, epoch)[-1]
wh_points = get_wh_points()
res_coeffs = stochasticGDAdaGrad(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
X = []
Y = []
print(res_coeffs, "<- res")
for i in get_wh_points():
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, wh_points, "Зависимость веса и роста 18-летних людей adaGrad со старым lr")
except FileNotFoundError:
  print("File wasn't uploaded")

wh_points = get_wh_points()
res_coeffs = stochasticGDAdaGrad(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]), lr=1.5)
X = []
Y = []
print(res_coeffs, "<- res")
for i in get_wh_points():
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, wh_points, "Зависимость веса и роста 18-летних людей adaGrad с увеличенным lr")
except FileNotFoundError:
  print("File wasn't uploaded")

linear_points = get_simple_points()
res_coeffs = stochasticGDAdaGrad(len(linear_points) // 2, linear_points, np.array([0.1, 0.1]), lr=1.5)
X = []
Y = []
print(res_coeffs, "<- res")
for i in linear_points:
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, linear_points, "Зависимость веса и роста 18-летних людей adaGrad с увеличенным lr")
except FileNotFoundError:
  print("File wasn't uploaded")


Здесь с lr из предыдущего шага алгоритм не работает так, как нужно. С увеличением learning rate константы всё работает.

RMSProp


In [ ]:
def stochasticGDRMS_all(batch_size, points, start_coeffs, lr=0.1, epoch = 100):
  i = 0
  gamma = 0.9
  coeffs = start_coeffs
  print(MSE(points, coeffs), "<- start MSE")
  V_prev=coeffs
  mse = []
  list_coeff = []
  while i < epoch:
    list_coeff.append(coeffs.copy())
    mse.append(MSE(points, coeffs))
    prev_coeffs = coeffs
    V = gamma * V_prev + (1 - gamma) * np.square(grad_MSE(points, prev_coeffs, batch_size))
    sum_prev = sum
    coeffs -= lr * np.divide(grad_MSE(points, prev_coeffs, batch_size), np.sqrt(V))
    random.shuffle(points)
    i += 1
  print(MSE(points, coeffs), "<- min MSE")
  fig = plt.figure()
  ax1 = fig.add_subplot(211)
  ax1.set_xlabel('epoch')
  ax1.plot(np.arange(epoch), mse)
  plt.show()
  res_coeffs = coeffs
  return list_coeff
def stochasticGDRMS(batch_size, points, start_coeffs, lr=0.1, epoch = 100):
  return stochasticGDRMS_all(batch_size, points, start_coeffs, lr, epoch)[-1]
wh_points = get_wh_points()
res_coeffs = stochasticGDRMS(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
X = []
Y = []
print(res_coeffs, "<- res")
for i in get_wh_points():
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, wh_points, "Зависимость веса и роста 18-летних людей RMSProp")
except FileNotFoundError:
  print("File wasn't uploaded")

Аналогично предыдущему методу: с увеличением константы всё работает, но с констанотой из первых 2-ух методов всё плохо :(

Adam (adaptive moment estimation)


In [ ]:
def stochasticAdam_all(batch_size, points, start_coeffs, lr=0.1, epoch = 100, f = MSE, grad = grad_MSE):
    i = 0
    gamma = 0.99
    coeffs = start_coeffs
    td_prev = start_coeffs
    beta = 0.9
    print(f(points, coeffs), "<- start MSE")
    V_prev=coeffs
    mse = []
    list_coeff = []
    while i < epoch:
      list_coeff.append(coeffs.copy())
      mse.append(f(points, coeffs))
      prev_coeffs = coeffs
      td = beta * td_prev + (1 - beta) * grad(points, prev_coeffs, batch_size)
      V = gamma * V_prev + (1 - gamma) * np.square(grad(points, prev_coeffs, batch_size))
      coeffs -= lr * np.divide(td, np.sqrt(V) + 1E-8)
      random.shuffle(points)
      i += 1
    print(f(points, coeffs), "<- min MSE")
    fig = plt.figure()
    ax1 = fig.add_subplot(211)
    ax1.set_xlabel('epoch')
    ax1.plot(np.arange(epoch), mse)
    plt.show()
    res_coeffs = coeffs
    return list_coeff
def stochasticAdam(batch_size, points, start_coeffs, lr=0.1, epoch = 100, f = MSE, grad = grad_MSE):
  return stochasticAdam_all(batch_size, points, start_coeffs, lr, epoch, f, grad)[-1]
wh_points = get_wh_points()
res_coeffs = stochasticAdam(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
X = []
Y = []
print(res_coeffs, "<- res")
for i in get_wh_points():
  X.append(i[0])

for i in X:
  Y.append(i * res_coeffs[0] + res_coeffs[1])

try:
  show_points_and_line(X, Y, wh_points, "Зависимость веса и роста 18-летних людей Adam")
except FileNotFoundError:
  print("File wasn't uploaded")

Аналогично предыдущему методу: с увеличением константы всё работает, но с констанотой из первых 2-ух методов всё плохо :(

## ***4. Сравнить методы.***

In [ ]:
print('SGD with momentum')
%time %memit stochasticGDmomentum(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
print('Nesterov')
%time %memit stochasticGDnesterov(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
print('AdaGrad')
%time %memit stochasticGDAdaGrad(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
print('RMSProp')
%time %memit stochasticGDRMS(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
print('Adam')
%time %memit stochasticAdam(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))

In [ ]:
import pandas as pd
'''df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
                  index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))
[["20 итераций", "надёжный", 189.13, "", 10],
                    ["20 итераций", "надёжный", 189.13, 10, 10],
                    ["5 итераций", "надёжный", 189.13, 10, 10],
                    ["в среднем 25", "ненадёжный", 189.13, 10, 10],
                    ["в среднем 25", "ненадёжный", 189.13, 10, 10]],
'''
df = pd.DataFrame([["20 итераций", "надёжный", 189.13, 42.7],
                    ["20 итераций", "надёжный", 189.13, 54],
                    ["5 итераций", "надёжный", 189.13, 48.6],
                    ["в среднем 25", "ненадёжный", 189.13, 48.4],
                    ["в среднем 25", "ненадёжный", 189.13, 49.6]],
                  index=pd.Index(['Momentum', 'Nesterov', 'AdaGrad', 'RMSProp', 'Adam'], name='Models:'),
                  columns=['Скорость сходимости', 'Надёжность', 'Объем оперативной памяти','Время выполнения'])
df.style

## ***5. График***

In [ ]:
space = np.linspace(-10, 10, 100)
x_def, y_def = np.meshgrid(space, space)
v_def = np.array([x_def, y_def])
padding = 5
def momentum():
  return stochasticGDmomentum_all(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
def nesterov():
  return stochasticGDnesterov_all(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
def adagrad():
  return stochasticGDAdaGrad_all(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]), lr=1.5)
def rmsprop():
  return stochasticGDRMS_all(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
def adam():
  return stochasticAdam_all(len(wh_points) // 2, wh_points, np.array([0.1, 0.1]))
def draw_graphic(f, array):
  plt.rcParams["figure.figsize"] = (20, 20)
  ogr = [float('inf'), float('-inf'), float('inf'), float('-inf'), float('inf'), float('-inf')]
  near_levels = []
  for coeffs in array:
    x_list = coeffs.transpose()[0]
    y_list = coeffs.transpose()[1]
    res = f(wh_points, v_def)
    this_ogr = [np.min(x_list), np.max(x_list), np.min(y_list), np.max(y_list), np.min(res), np.max(res)]
    for i in range(6):
      if i%2 == 0:
        if (this_ogr[i] < ogr[i]):
          ogr[i] = this_ogr[i]
      else:
        if (this_ogr[i] > ogr[i]):
          ogr[i] = this_ogr[i]

    '''horisontal_size = max_x - min_x
    vertical_size = max_y - min_y
    space_x = np.linspace(min_x-horisontal_size/4, max_x+horisontal_size/4, int(100))
    space_y = np.linspace(min_y-vertical_size/4, max_y+vertical_size/4, int(100))
    space = np.linspace(-10, 10, 100)
    x, y = np.meshgrid(space_x, space_y)
    v = np.array([x, y])
    res = f(wh_points, v)
    minf = np.min(res)
    maxf = np.max(res)'''
    near_levels += [f(wh_points, p) for p in coeffs[::3]]
    plt.plot(coeffs[:, 0], coeffs[:, 1], 'o-')
    '''ax = plt.figure().add_subplot()
    ax.plot(coeffs[:, 0], coeffs[:, 1], 'o-')
    print([f(wh_points, p) for p in coeffs] )
    ax.contour(x, y, f(wh_points, v), levels = np.unique(sorted(list(np.linspace(minf, maxf, 100)))))''' #[f(wh_points, p) for p in coeffs] +
  final_ogr = [max(-10, ogr[0]) - ogr[0] / padding, min(10, ogr[1]) - ogr[1] / padding, max(-10, ogr[2]) - ogr[2] / padding, min(10, ogr[3]) + ogr[3] / padding]
  space_x = np.linspace(final_ogr[0], final_ogr[1], 100)
  space_y = np.linspace(final_ogr[2], final_ogr[3], 100)
  x, y = np.meshgrid(space_x, space_y)
  v = np.array([x, y])
  plt.xlim(final_ogr[0], final_ogr[1])
  plt.ylim(final_ogr[2], final_ogr[3])
  plt.contour(x, y, f(wh_points, v), levels = np.unique(sorted(near_levels + list(np.linspace(ogr[4], ogr[5], 1000)))))
  plt.show()

draw_graphic(MSE, np.array([momentum(), nesterov(), adagrad(), rmsprop(), adam()]))
'''
adam - фиолетовый, 
rmsprop - красный, 
adagrad - зелёный, 
nesterov - оранжевый, 
momentum - синий'''